In [2]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
# nltk.download('punkt')
import json

In [2]:
with open('./Dynamically-Generated-Hate-Speech-Dataset/Dynamically Generated Hate Dataset v0.2.3.csv', 'r') as f:
    data = pd.read_csv(f)
    

In [3]:
data.head()

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched
0,1,acl1,1,dalits are lowlives,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
1,2,acl10,2,gay people are a burden to our society,hate,notgiven,notgiven,original,train,1,GNZuCtwed3,1,NaN
2,3,acl100,3,Arabs are not welcome,hate,notgiven,notgiven,original,train,1,vDe7GN0NrL,1,NaN
3,4,acl1000,4,I'm not saying we should actually eliminate he...,hate,notgiven,notgiven,original,train,1,oemYWm1Tjg,1,NaN
4,5,acl10000,5,bananas are for black people,hate,notgiven,notgiven,original,test,1,QiOKkCi7F8,1,NaN


In [126]:
list_of_targets = list(set(data['target']))
list_of_targets = list(set([x for y in list_of_targets for x in str(y).split(', ')]))

list_of_targets.remove('none')


splitting_classes = ['asi.wom','bla.wom','non.white.wom','gay.man','asi.man','indig.wom','mus.wom','bla.man','gay.wom']
list_of_targets_flat = []
for t in list_of_targets:
    if t in splitting_classes:
        if len(t.split('.')) > 2:
            list_of_targets_flat += ['.'.join(t.split('.')[:-1]), t.split('.')[-1]]
        else:
            list_of_targets_flat += t.split('.')
    else:
        list_of_targets_flat.append(t)
list_of_targets_flat = list(set(list_of_targets_flat))

NUM_OF_CLASSES = len(list_of_targets_flat)


In [20]:
mus_only = data[data['target'].str.contains('mus')==True]
jew_only = data[data['target'].str.contains('jew')==True]

In [127]:
# building vocab

all_sents = list(data['text'])
all_tokenized_sents = [word_tokenize(s) for s in all_sents]

ps = PorterStemmer()

all_stemmed_sents = [[ps.stem(w) for w in s] for s in all_tokenized_sents]
vocab = list(set([word for s in all_stemmed_sents for word in s]))

VOCAB_SIZE = len(vocab)

In [128]:
likelihood = np.zeros((VOCAB_SIZE, NUM_OF_CLASSES))
prior = np.zeros((NUM_OF_CLASSES,1))

In [129]:
# ps = PorterStemmer()

for c_pinter, c in enumerate(list_of_targets_flat):
    # stem subset
    subset = data[data['target'].str.contains(c)==True]
    prior[c_pinter,0] = len(subset)/len(data)
    sentences = list(subset['text'])
    sub_tokenized_sents = [word_tokenize(s) for s in sentences]
    sub_stemmed_sents = [[ps.stem(w) for w in s] for s in sub_tokenized_sents]
    sub_stemmed_flatten = [x for y in sub_stemmed_sents for x in y]
    
    
    counts = {}
    for w in sub_stemmed_flatten:
        if w in counts:
            counts[w] += 1
        else:
            counts[w] = 1

    for w_pointer, w in enumerate(vocab):
        if w in counts:
            likelihood[w_pointer, c_pinter] = counts[w] / len(sub_stemmed_flatten)

In [130]:
posterior_prop = likelihood * np.repeat(prior,VOCAB_SIZE,axis=1).T

In [136]:
most_likely_class_for_word = np.argmax(posterior_prop, axis=1)

In [150]:
list_of_targets_flat.index('jew')

3

In [151]:
likely_target_class_words = list(np.where(most_likely_class_for_word == 3)[0])

In [152]:
for i in likely_target_class_words:
    print(vocab[i])

host
second
la
control
fellow
driven
destroy
)
nationalist
iran
spell
censor
behind
debt
ring
historian
outcom
leftist
israel
corrupt
(
war
mirror
wasnt


In [61]:
# stem subset
sentences = list(mus_only['text'])
sub_tokenized_sents = [word_tokenize(s) for s in sentences]

ps = PorterStemmer()

sub_stemmed_sents = [[ps.stem(w) for w in s] for s in sub_tokenized_sents]

In [64]:
sub_stemmed_flatten = [x for y in sub_stemmed_sents for x in y]

In [68]:
counts = {}
for w in sub_stemmed_flatten:
    if w in counts:
        counts[w] += 1
    else:
        counts[w] = 1
        
likelihood = {}
for w, c in counts.items():
    likelihood[w] = c / len(sub_stemmed_flatten)

In [69]:
likelihood

{'can': 0.002967288610359398,
 'you': 0.010088781275221953,
 'smell': 0.00011869154441437592,
 'these': 0.0021127094905758915,
 'dirti': 0.00023738308882875184,
 'muzzi': 0.0009732706641978825,
 '?': 0.00576840905853867,
 'it': 0.013127284812229977,
 "'s": 0.006979062811565304,
 'a': 0.018302236148696766,
 'brutal': 0.00026112139771162704,
 'stank': 2.3738308882875185e-05,
 'there': 0.0036319612590799033,
 'are': 0.013151023121112852,
 'absolut': 0.00047476617765750367,
 'load': 0.00011869154441437592,
 'of': 0.019892702843849405,
 'filthi': 0.0001424298532972511,
 'muslim': 0.020391207330389783,
 'on': 0.004700185158809287,
 'my': 0.003726914494611404,
 'street': 0.0004035512510088781,
 'whi': 0.002160186108341642,
 'is': 0.01592840526040925,
 'such': 0.000902055737549257,
 'high': 0.00016616816218012627,
 'concentr': 7.121492664862555e-05,
 'here': 0.0014480368418553862,
 'compar': 0.0001424298532972511,
 'to': 0.02331101932298343,
 'normal': 0.00035607463324312775,
 'peopl': 0.00548

In [4]:
arr = np.array([[.5, .5], [.9, .1], [.8, .2]])

In [5]:
arr

array([[0.5, 0.5],
       [0.9, 0.1],
       [0.8, 0.2]])

In [13]:
i = np.where(arr.max(axis=1) >= 0.5)[0]

In [25]:
a = np.ones([5,1])
np.repeat (a, 6, axis=1)

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])

In [21]:
def stable_softmax(x):
    z = x - max(x)
    numerator = np.exp(z)
    denominator = np.sum(numerator)
    softmax = numerator/denominator

    return softmax
a = np.array([0.000001, 0])
a / np.array([0.00000001])

array([100.,   0.])